3. april 2016

'wir sind die coolsten wenn wir cruisen wenn wir durch die city düsen'

- Andre Lenz, Sommer 2009


Alle daten holen von theaiGames.com 

- verbindung mit urllib
- datenholen mit beautiful soup
- abspeichern in pandas dataframe mit multiindex
- exportieren als pickle zur wiederverwendung



In [60]:
from IPython.core.display import HTML
HTML("<style>.container { width:90% !important; }</style>")

In [61]:
import pandas as pd
from pandas.io.json import json_normalize
import numpy as npuuuuuuu
import urllib
from bs4 import BeautifulSoup
import pylab

In [82]:
frames = {}
sites = [] #collection of games links
playerLinks = [] # collect links of specific player, temp
player1 = []
player2 = []
playerTags = []
desiredBots = ['Jyzaya_Ultimate']
numberOfSites = 5 #7 # how many pages of games to download

In [84]:
%%time
#Get the Bots game links

for bot in desiredBots:
    
    for i in range(1, numberOfSites):
        print bot + str(i)
        connectionString = 'http://theaigames.com/competitions/ultimate-tic-tac-toe/game-log/'+ bot + '/' + str(i)
        try:
            connection = urllib.urlopen(connectionString)
            soup = BeautifulSoup(connection, "lxml")
            for link in soup.findAll('a'): 
                link = link.get('href')
                if link.startswith('http://theaigames.com/competitions/ultimate-tic-tac-toe/games/') == True:
                    playerLinks.append(link) #write all game links to one array

            
            if len(playerLinks) == 0:
                break
            playerTagsOnSite = soup.find_all(attrs={'class': 'div-botName-gameLog'})
            playerTags = playerTags + playerTagsOnSite
            sites = sites + playerLinks
            playerLinks = []

        except:
            print "Conection failed for bot and Site: " + bot + '/' + str(i)
        finally:
            connection.close
        
            
#10, 7 sec mit 2 seiten und 1 boti



Jyzaya_Ultimate1
Jyzaya_Ultimate2
Jyzaya_Ultimate3
Jyzaya_Ultimate4
CPU times: user 188 ms, sys: 4 ms, total: 192 ms
Wall time: 2min 28s


In [86]:
# differentiate for player 1 or 2 in a list with same lenght as game link list
c = 0
for playerTag in playerTags:
    if (c%2 == 0):
        tempPlayer = playerTag.getText('div-botName-gameLog')
        tempPlayer = tempPlayer.strip()
        player1.append(tempPlayer)
    else:
        tempPlayer = playerTag.getText('div-botName-gameLog')
        tempPlayer = tempPlayer.strip()
        player2.append(tempPlayer)
    c += 1

In [91]:
player1?

In [96]:
sites

['http://theaigames.com/competitions/ultimate-tic-tac-toe/games/570400ab4ee9c61ae0b9cde9',
 'http://theaigames.com/competitions/ultimate-tic-tac-toe/games/5703fafd4ca80140ee32cbba',
 'http://theaigames.com/competitions/ultimate-tic-tac-toe/games/5703dd634ca80140ee32cb01',
 'http://theaigames.com/competitions/ultimate-tic-tac-toe/games/5703c9a74ee9c61ae0b9cbcb',
 'http://theaigames.com/competitions/ultimate-tic-tac-toe/games/57033bb14ee9c644ff451f8b',
 'http://theaigames.com/competitions/ultimate-tic-tac-toe/games/570303964ee9c644ff451d68',
 'http://theaigames.com/competitions/ultimate-tic-tac-toe/games/5702ff6a4ca801276beb4dd3',
 'http://theaigames.com/competitions/ultimate-tic-tac-toe/games/5702ecb24ca801276beb4d0b',
 'http://theaigames.com/competitions/ultimate-tic-tac-toe/games/5702e95a4ee9c644ff451c6a',
 'http://theaigames.com/competitions/ultimate-tic-tac-toe/games/5702dd934ee9c644ff451bfc',
 'http://theaigames.com/competitions/ultimate-tic-tac-toe/games/5702a0634ee9c644ff4519a8',

In [133]:
sites[:1]

['http://theaigames.com/competitions/ultimate-tic-tac-toe/games/570400ab4ee9c61ae0b9cde9']

In [111]:
# spieldaten holen von webseite
%time

for site in sites[:1]: # pro game etwa 8 secunden
    print "1"
    site = site + '/data'
    print "2"
    ttt = pd.read_json(site, typ = 'series')
    print "3"
    ttt = json_normalize(ttt['states'])
    print "4"
    ttt = ttt.replace('', np.nan)
    print "5"
    ttt.winner =ttt.winner.fillna(method = 'bfill')
    ttt.illegalMove = ttt.illegalMove.fillna('')
    print "6"
    frames[site] = ttt
    print "game done"
#games = pd.concat(frames)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.01 µs
1
2
3
4
5
6
game done


In [118]:
games = pd.concat(frames)

In [132]:
frames

{'http://theaigames.com/competitions/ultimate-tic-tac-toe/games/5702a0634ee9c644ff4519a8/data':                                                  field illegalMove  move  \
 0    4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,...                 1   
 1    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...                 1   
 2    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...                 2   
 3    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...                 2   
 4    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...                 3   
 5    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...                 3   
 6    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...                 4   
 7    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...                 4   
 8    0,0,0,4,4,4,0,0,0,0,0,0,4,4,4,0,0,0,0,0,0,4,4,...                 5   
 9    0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...                 5   
 10   0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...   

In [130]:
games.index.levels[0]

Index([u'http://theaigames.com/competitions/ultimate-tic-tac-toe/games/5702a0634ee9c644ff4519a8/data',
       u'http://theaigames.com/competitions/ultimate-tic-tac-toe/games/5702dd934ee9c644ff451bfc/data',
       u'http://theaigames.com/competitions/ultimate-tic-tac-toe/games/5702e95a4ee9c644ff451c6a/data',
       u'http://theaigames.com/competitions/ultimate-tic-tac-toe/games/5702ecb24ca801276beb4d0b/data',
       u'http://theaigames.com/competitions/ultimate-tic-tac-toe/games/5702ff6a4ca801276beb4dd3/data',
       u'http://theaigames.com/competitions/ultimate-tic-tac-toe/games/570303964ee9c644ff451d68/data',
       u'http://theaigames.com/competitions/ultimate-tic-tac-toe/games/57033bb14ee9c644ff451f8b/data',
       u'http://theaigames.com/competitions/ultimate-tic-tac-toe/games/5703c9a74ee9c61ae0b9cbcb/data',
       u'http://theaigames.com/competitions/ultimate-tic-tac-toe/games/5703dd634ca80140ee32cb01/data',
       u'http://theaigames.com/competitions/ultimate-tic-tac-toe/games/57

In [ ]:
# abschpeichern als pickle in windows oder linux
games.to_pickle('games.pkl')
#games.to_pickle('C:\Users\User\Desktop\Statistics\TicTacToe\games.pkl')